In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import r2_score, accuracy_score, auc, roc_curve, confusion_matrix, classification_report
from imblearn.metrics import classification_report_imbalanced

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate, KFold
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.model_selection import cross_val_score

from catboost import CatBoostClassifier, Pool

In [2]:
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel
# from skater.core.global_interpretation.tree_surrogate import TreeSurrogate
# from skater.util.dataops import show_in_notebook

In [3]:
url='https://drive.google.com/file/d/1LOjJxnQIjf3Jp01Th2GYa8Q0kAYL8Uq7/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]

df = pd.read_csv(url)

df

,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,RainToday,Month,Week_Number,Day_Number,Location,WindGustDir,WindDir9am,WindDir3pm,Koppen climate classification,RainTomorrow
0,13.4,22.9,0.6,4.8,8.5,44.0,20.0,24.0,71.0,22.0,...,0.0,12.0,49.0,336.0,Albury,W,W,WNW,cfa,0.0
1,7.4,25.1,0.0,4.8,8.5,44.0,4.0,22.0,44.0,25.0,...,0.0,12.0,49.0,337.0,Albury,WNW,NNW,WSW,cfa,0.0
2,12.9,25.7,0.0,4.8,8.5,46.0,19.0,26.0,38.0,30.0,...,0.0,12.0,49.0,338.0,Albury,WSW,W,WSW,cfa,0.0
3,9.2,28.0,0.0,4.8,8.5,24.0,11.0,9.0,45.0,16.0,...,0.0,12.0,49.0,339.0,Albury,NE,SE,E,cfa,0.0
4,17.5,32.3,1.0,4.8,8.5,41.0,7.0,20.0,82.0,33.0,...,0.0,12.0,49.0,340.0,Albury,W,ENE,NW,cfa,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142188,3.5,21.8,0.0,4.8,8.5,31.0,15.0,13.0,59.0,27.0,...,0.0,6.0,25.0,171.0,Uluru,E,ESE,E,bwh,0.0
142189,2.8,23.4,0.0,4.8,8.5,31.0,13.0,11.0,51.0,24.0,...,0.0,6.0,25.0,172.0,Uluru,E,SE,ENE,bwh,0.0
142190,3.6,25.3,0.0,4.8,8.5,22.0,13.0,9.0,56.0,21.0,...,0.0,6.0,25.0,173.0,Uluru,NNW,SE,N,bwh,0.0
142191,5.4,26.9,0.0,4.8,8.5,37.0,9.0,9.0,53.0,24.0,...,0.0,6.0,25.0,174.0,Uluru,N,SE,WNW,bwh,0.0


In [11]:
# 

# raplace location column by climate class
# reduce dimension of wind columns

df_4 = df
df_4 = df_4.drop('Location', axis=1)

# Koppen climate classification
df_4 = df_4.join(pd.get_dummies(df_4['Koppen climate classification'], prefix='climate_clas', prefix_sep='_', drop_first=True))
df_4 = df_4.drop('Koppen climate classification', axis=1)

# WindGustDir_N
df_4['WindGustDir_N']=0
df_4.loc[ (df_4['WindGustDir'] == 'N'), ['WindGustDir_N']] = '4'
df_4.loc[ (df_4['WindGustDir'] == 'NNW') | (df_4['WindGustDir'] == 'NNE'), ['WindGustDir_N']] = '3'
df_4.loc[ (df_4['WindGustDir'] == 'NW') | (df_4['WindGustDir'] == 'NE'), ['WindGustDir_N']] = '2'
df_4.loc[ (df_4['WindGustDir'] == 'WNW') | (df_4['WindGustDir'] == 'ENE'), ['WindGustDir_N']] = '1'

# WindGustDir_S
df_4['WindGustDir_S']=0
df_4.loc[ (df_4['WindGustDir'] == 'S'), ['WindGustDir_S']] = '4'
df_4.loc[ (df_4['WindGustDir'] == 'SSW') | (df_4['WindGustDir'] == 'SSE'), ['WindGustDir_S']] = '3'
df_4.loc[ (df_4['WindGustDir'] == 'SW') | (df_4['WindGustDir'] == 'SE'), ['WindGustDir_S']] = '2'
df_4.loc[ (df_4['WindGustDir'] == 'WSW') | (df_4['WindGustDir'] == 'ESE'), ['WindGustDir_S']] = '1'

# WindGustDir_W
df_4['WindGustDir_W']=0
df_4.loc[ (df_4['WindGustDir'] == 'W'), ['WindGustDir_W']] = '4'
df_4.loc[ (df_4['WindGustDir'] == 'WNW') | (df_4['WindGustDir'] == 'WSW'), ['WindGustDir_W']] = '3'
df_4.loc[ (df_4['WindGustDir'] == 'NW') | (df_4['WindGustDir'] == 'SW'), ['WindGustDir_W']] = '2'
df_4.loc[ (df_4['WindGustDir'] == 'NNW') | (df_4['WindGustDir'] == 'SSW'), ['WindGustDir_W']] = '1'

# WindGustDir_E
df_4['WindGustDir_E']=0
df_4.loc[ (df_4['WindGustDir'] == 'E'), ['WindGustDir_E']] = '4'
df_4.loc[ (df_4['WindGustDir'] == 'ENE') | (df_4['WindGustDir'] == 'ESE'), ['WindGustDir_E']] = '3'
df_4.loc[ (df_4['WindGustDir'] == 'NE') | (df_4['WindGustDir'] == 'SE'), ['WindGustDir_E']] = '2'
df_4.loc[ (df_4['WindGustDir'] == 'NNE') | (df_4['WindGustDir'] == 'SSE'), ['WindGustDir_E']] = '1'
df_4 = df_4.drop('WindGustDir', axis=1)

# WindDir9am_N
df_4['WindDir9am_N']=0
df_4.loc[ (df_4['WindDir9am'] == 'N'), ['WindDir9am_N']] = '4'
df_4.loc[ (df_4['WindDir9am'] == 'NNW') | (df_4['WindDir9am'] == 'NNE'), ['WindDir9am_N']] = '3'
df_4.loc[ (df_4['WindDir9am'] == 'NW') | (df_4['WindDir9am'] == 'NE'), ['WindDir9am_N']] = '2'
df_4.loc[ (df_4['WindDir9am'] == 'WNW') | (df_4['WindDir9am'] == 'ENE'), ['WindDir9am_N']] = '1'

# WindDir9am_S
df_4['WindDir9am_S']=0
df_4.loc[ (df_4['WindDir9am'] == 'S'), ['WindDir9am_S']] = '4'
df_4.loc[ (df_4['WindDir9am'] == 'SSW') | (df_4['WindDir9am'] == 'SSE'), ['WindDir9am_S']] = '3'
df_4.loc[ (df_4['WindDir9am'] == 'SW') | (df_4['WindDir9am'] == 'SE'), ['WindDir9am_S']] = '2'
df_4.loc[ (df_4['WindDir9am'] == 'WSW') | (df_4['WindDir9am'] == 'ESE'), ['WindDir9am_S']] = '1'

# WindDir9am_W
df_4['WindDir9am_W']=0
df_4.loc[ (df_4['WindDir9am'] == 'W'), ['WindDir9am_W']] = '4'
df_4.loc[ (df_4['WindDir9am'] == 'WNW') | (df_4['WindDir9am'] == 'WSW'), ['WindDir9am_W']] = '3'
df_4.loc[ (df_4['WindDir9am'] == 'NW') | (df_4['WindDir9am'] == 'SW'), ['WindDir9am_W']] = '2'
df_4.loc[ (df_4['WindDir9am'] == 'NNW') | (df_4['WindDir9am'] == 'SSW'), ['WindDir9am_W']] = '1'

# WindDir9am_E
df_4['WindDir9am_E']=0
df_4.loc[ (df_4['WindDir9am'] == 'E'), ['WindDir9am_E']] = '4'
df_4.loc[ (df_4['WindDir9am'] == 'ENE') | (df_4['WindDir9am'] == 'ESE'), ['WindDir9am_E']] = '3'
df_4.loc[ (df_4['WindDir9am'] == 'NE') | (df_4['WindDir9am'] == 'SE'), ['WindDir9am_E']] = '2'
df_4.loc[ (df_4['WindDir9am'] == 'NNE') | (df_4['WindDir9am'] == 'SSE'), ['WindDir9am_E']] = '1'
df_4 = df_4.drop('WindDir9am', axis=1)

# WindDir3pm_N
df_4['WindDir3pm_N']=0
df_4.loc[ (df_4['WindDir3pm'] == 'N'), ['WindDir3pm_N']] = '4'
df_4.loc[ (df_4['WindDir3pm'] == 'NNW') | (df_4['WindDir3pm'] == 'NNE'), ['WindDir3pm_N']] = '3'
df_4.loc[ (df_4['WindDir3pm'] == 'NW') | (df_4['WindDir3pm'] == 'NE'), ['WindDir3pm_N']] = '2'
df_4.loc[ (df_4['WindDir3pm'] == 'WNW') | (df_4['WindDir3pm'] == 'ENE'), ['WindDir3pm_N']] = '1'

# WindDir3pm_S
df_4['WindDir3pm_S']=0
df_4.loc[ (df_4['WindDir3pm'] == 'S'), ['WindDir3pm_S']] = '4'
df_4.loc[ (df_4['WindDir3pm'] == 'SSW') | (df_4['WindDir3pm'] == 'SSE'), ['WindDir3pm_S']] = '3'
df_4.loc[ (df_4['WindDir3pm'] == 'SW') | (df_4['WindDir3pm'] == 'SE'), ['WindDir3pm_S']] = '2'
df_4.loc[ (df_4['WindDir3pm'] == 'WSW') | (df_4['WindDir3pm'] == 'ESE'), ['WindDir3pm_S']] = '1'

# WindDir3pm_W
df_4['WindDir3pm_W']=0
df_4.loc[ (df_4['WindDir3pm'] == 'W'), ['WindDir3pm_W']] = '4'
df_4.loc[ (df_4['WindDir3pm'] == 'WNW') | (df_4['WindDir3pm'] == 'WSW'), ['WindDir3pm_W']] = '3'
df_4.loc[ (df_4['WindDir3pm'] == 'NW') | (df_4['WindDir3pm'] == 'SW'), ['WindDir3pm_W']] = '2'
df_4.loc[ (df_4['WindDir3pm'] == 'NNW') | (df_4['WindDir3pm'] == 'SSW'), ['WindDir3pm_W']] = '1'

# WindDir3pm_E
df_4['WindDir3pm_E']=0
df_4.loc[ (df_4['WindDir3pm'] == 'E'), ['WindDir3pm_E']] = '4'
df_4.loc[ (df_4['WindDir3pm'] == 'ENE') | (df_4['WindDir3pm'] == 'ESE'), ['WindDir3pm_E']] = '3'
df_4.loc[ (df_4['WindDir3pm'] == 'NE') | (df_4['WindDir3pm'] == 'SE'), ['WindDir3pm_E']] = '2'
df_4.loc[ (df_4['WindDir3pm'] == 'NNE') | (df_4['WindDir3pm'] == 'SSE'), ['WindDir3pm_E']] = '1'
df_4 = df_4.drop('WindDir3pm', axis=1)

# 
data = df_4.drop('RainTomorrow', axis=1)
target = df_4.RainTomorrow

#
features = list(data.columns)

# train test split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.1)

# StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
df_4.columns

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am',
       'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm',
       'Temp9am', 'Temp3pm', 'RainToday', 'Month', 'Week_Number', 'Day_Number',
       'RainTomorrow', 'climate_clas_aw', 'climate_clas_bsk',
       'climate_clas_bwh', 'climate_clas_cfa', 'climate_clas_cfb',
       'climate_clas_csa', 'climate_clas_csb', 'WindGustDir_N',
       'WindGustDir_S', 'WindGustDir_W', 'WindGustDir_E', 'WindDir9am_N',
       'WindDir9am_S', 'WindDir9am_W', 'WindDir9am_E', 'WindDir3pm_N',
       'WindDir3pm_S', 'WindDir3pm_W', 'WindDir3pm_E'],
      dtype='object')

In [13]:
catboost_1 = CatBoostClassifier(iterations=5000, verbose=False, random_state=0, task_type="GPU", devices='0:1', scale_pos_weight=2) 

catboost_1.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)

y_pred = catboost_1.predict(X_test)

cm = confusion_matrix(y_test,y_pred)
print(cm)

print(classification_report_imbalanced(y_test, y_pred))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

[[9897 1084]
 [ 914 2325]]
                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.92      0.90      0.72      0.91      0.80      0.66     10981
        1.0       0.68      0.72      0.90      0.70      0.80      0.64      3239

avg / total       0.86      0.86      0.76      0.86      0.80      0.65     14220



In [14]:
# Save the catboost model
catboost_1.save_model(fname='catboost_model')

In [15]:
# Load the model
# from_file = CatBoostClassifier()
# from_file.load_model("catboost_model")

In [16]:
# Save the scaler
from joblib import dump, load
dump(scaler, 'catboost_std_scaler.bin', compress=True)

['catboost_std_scaler.bin']

In [17]:
# Load the scaler
# sc=load('catboost_std_scaler.bin')